In [1]:
import torch
import torch.nn as nn
import os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision.transforms import v2 as transforms
import torchvision
import cv2
import torch.nn as nn
from torch.autograd import Variable
from torch.nn.utils import spectral_norm
import time as t
#plt.switch_backend('agg')
from torchvision import utils
import tensorflow as tf
from torch import autograd
from itertools import chain

In [2]:
def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        m.weight.data.normal_(0.0, 0.02)
    elif classname.find('BatchNorm') != -1:
        m.weight.data.normal_(1.0, 0.02)
        m.bias.data.fill_(0)

In [3]:
def compute_acc(preds, labels):
    correct = 0
    preds_ = preds.data.max(1)[1]
    correct = preds_.eq(labels.data).cpu().sum()
    acc = float(correct) / float(len(labels.data)) * 100.0
    return acc

In [4]:
class GaussianNoise(nn.Module):
    def __init__(self, std=0.1, decay_rate=0):
        super().__init__()
        self.std = std
        self.decay_rate = decay_rate

    def decay_step(self):
        self.std = max(self.std*self.decay_rate, 0)

    def forward(self, x):
        if self.training:
            noise = torch.empty_like(x).normal_(std=self.std)
            batch_size = noise.size(0)    
            # Randomly select half of the indices
            indices_to_zero = torch.randperm(batch_size)[:batch_size // 2]
            # Zero out the selected half
            noise[indices_to_zero] = 0
            return x + noise
        else:
            return x

In [5]:
def decay_gauss_std(net):
    std = 0
    for m in net.modules():
        if isinstance(m, GaussianNoise):
            m.decay_step()
            std = m.std

In [6]:
class Generator(nn.Module):
    def __init__(self, n_channels, z_dim, n_classes, n_embedding, std, decay):
        super().__init__()
        self.std = std
        self.std_decay_rate = decay
        self.embedding = nn.Sequential(nn.Embedding(n_classes, z_dim),
                                       GaussianNoise(self.std, self.std_decay_rate),)
        #self.latent_class = n_classes*n_embedding
        self.z_dim = z_dim
        #self.fc_emb = nn.Linear(n_embedding, z_dim, bias=False)
        self.fc_latent = nn.Sequential(#GaussianNoise(self.std, self.std_decay_rate),
                                       nn.Linear(z_dim, 400, bias=False),)
        # IN ---> latent_space = 500
        self.block_c1 = self.generate_block(400, 256,4,4,0)
        self.block_c2 = self.generate_block(256, 128,8,4,2)
        self.block_c3 = self.generate_block(128, 64, 8,4,2)
        self.block_c4 = nn.Sequential(nn.ConvTranspose2d(64, n_channels, 4 , (2,3), (1,14)), 
                                      nn.Tanh(),)
        # OUT--> image_shape = (128,165,3)

    def generate_block(self, filter_in, filter_out, k_size, stride, padding):
    	return nn.Sequential(nn.ConvTranspose2d(filter_in, filter_out, k_size, stride, padding, bias=False),
            			     nn.BatchNorm2d(filter_out),
            			     nn.ReLU(True),)

    def forward(self, z, c):
        # Embedding transformation
        c = self.embedding(c).unsqueeze(2).unsqueeze(3)
        z = torch.mul(z,c)
        #print(z.shape)

        # Latent space transformation
        z = z.view(-1, self.z_dim) #could be squeeze(2).squeeze(3)
        z = nn.functional.relu(self.fc_latent(z))
        z = z.unsqueeze(2).unsqueeze(3)
                
        # Generate PRPD fake data
        x = self.block_c1(z)
        x = self.block_c2(x)
        x = self.block_c3(x)
        return self.block_c4(x)

In [7]:
class Discriminator(nn.Module):
    def __init__(self, n_channels, n_classes, std, decay):
        super().__init__()
        in_size = np.array((165,128))
        self.std = std
        self.std_decay_rate = decay
        # Convolutional layers
        self.block_c1 = self.generate_block(n_channels, 64, 8, 4, 2)
        self.out_size = self.compute_output_size(in_size, 8, 4, 2)
        self.block_c2 = self.generate_block(64, 128, 8, 4, 2)
        self.out_size = self.compute_output_size(self.out_size, 8, 4, 2)
        self.block_c3 = self.generate_block(128, 256, 4, 2, 1)
        #self.out_size = self.compute_output_size(self.out_size, 4, 2, 1)
        #self.block_c4 = self.generate_block(256,512, 4, 2, 1)
        self.out_size = np.prod(self.compute_output_size(self.out_size, 4, 2, 1))
        # Fully-connected layers
        self.block_dis = nn.Sequential(GaussianNoise(self.std, self.std_decay_rate),
                                       nn.Linear(self.out_size*256,1),
                                       nn.Sigmoid(),)
        self.block_aux = nn.Sequential(GaussianNoise(self.std, self.std_decay_rate),
                                       nn.Linear(self.out_size*256, n_classes),
                                       nn.Softmax(dim=1),)

    def compute_output_size(self, input_size, kernel_size, stride, padding=0, dilation=1):
        return ((input_size + 2*padding - dilation*(kernel_size - 1) - 1) // stride) + 1
    
    def generate_block(self, filter_in, filter_out, k_size, stride, padding):
        return nn.Sequential(GaussianNoise(self.std, self.std_decay_rate),
                             spectral_norm(nn.Conv2d(filter_in, filter_out, k_size, stride, padding, bias=False)),
                             nn.LeakyReLU(0.2, True),
                             nn.Dropout(0.5, inplace=False),)

    def forward(self, x):
        # image go through convolutional layers
        x = self.block_c1(x)
        x = self.block_c2(x)
        x = self.block_c3(x)
        #x = self.block_c4(x)
        x = x.view(-1, self.out_size*256)
        # go through MLP layers
        x_dis = self.block_dis(x)
        x_aux = self.block_aux(x)

        return (x_dis, x_aux)

In [8]:
class ACGAN(object):
    def __init__(self, n_channels, z_dim, n_classes, n_embedding, epochs, batch_size, device, cuda, std, decay):
        print('ACGAN Model Initialization')
        # Model parameters
        self.n_channels = n_channels
        self.z_dim = z_dim
        self.n_classes = n_classes
        self.n_embedding = n_embedding
        self.epochs = epochs
        self.batch_size = batch_size
        self.std = std
        self.decay = decay
        # Generator - Discriminator
        self.G = Generator(n_channels, z_dim, n_classes, n_embedding, std, decay)
        self.print_network(self.G, 'Generator')
        self.G.apply(weights_init)
        self.D = Discriminator(n_channels, n_classes, std, decay)
        self.D.apply(weights_init)
        self.print_network(self.D, 'Discriminator')
        self.device = device
        self.cuda = False
        self.cuda_index = 0
        
        # WGAN values from paper
        self.learning_rate = 1e-4
        self.b1 = 0
        self.b2 = 0.999 #according to paper         

        # Set the logger
        #self.logger = Logger('./logs-PRPD')
        #self.logger.writer.flush()
        self.number_of_images = 10
        self.lambda_term = 20

        # loss functions
        self.dis_criterion = nn.BCELoss()
        self.aux_criterion = nn.NLLLoss()
        self.check_cuda(cuda)

        self.eval_noise = torch.randn((self.batch_size, self.z_dim, 1, 1))#np.random.normal(0, 1, (self.batch_size, self.z_dim))#torch.rand((self.batch_size, 128, 1, 1))
        self.eval_noise = self.get_torch_variable(self.eval_noise)

        self.eval_label = torch.randint(0, n_classes, (batch_size,))
        self.eval_label = self.get_torch_variable(self.eval_label)

        # Optimizer
        self.d_optimizer = torch.optim.Adam(self.D.parameters(), lr=self.learning_rate, betas=(self.b1, self.b2))
        self.g_optimizer = torch.optim.Adam(self.G.parameters(), lr=self.learning_rate, betas=(self.b1, self.b2))

        # Variables
        self.G_error = np.zeros((self.epochs))
        self.D_error = np.zeros((self.epochs))
        self.D_x = np.zeros((self.epochs))
        self.D_G_z1 = np.zeros((self.epochs))
        self.D_G_z2 = np.zeros((self.epochs))
        self.accuracy = np.zeros((self.epochs))
        self.avg_loss_D = np.zeros((self.epochs))
        self.avg_loss_G = np.zeros((self.epochs))
        self.avg_accuracy = np.zeros((self.epochs))
    
    def check_cuda(self, cuda_flag=False):
        if cuda_flag:
            self.cuda = True
            self.D.cuda(self.cuda_index)
            self.G.cuda(self.cuda_index)
            self.dis_criterion = nn.BCELoss().cuda(self.cuda_index)
            self.aux_criterion = nn.NLLLoss().cuda(self.cuda_index)
            print("Cuda enabled flag: ")
            print(self.cuda)

    def get_torch_variable(self, arg):
        if self.cuda:
            return Variable(arg).cuda(self.cuda_index)
        else:
            return Variable(arg)

    def evaluate(self):
        self.D.eval()
        self.G.eval()
        avg_errD = 0.0
        avg_Dx   = 0.0
        avg_acc  = 0.0
        for i, batch in enumerate(val_loader):
            imgs, labels = batch
            if len(images) != self.batch_size:
                continue

            images, aux_label = self.get_torch_variable(imgs), self.get_torch_variable(labels)
            dis_label = self.get_torch_variable(torch.zeros((self.batch_size,)).fill_(1))

            # Enter real images
            dis_out, aux_out = self.D(images)
            dis_errD_real = self.dis_criterion(dis_out.squeeze(1), dis_label)
            aux_errD_real = self.aux_criterion(aux_out.squeeze(1), aux_label)
            errD_real = dis_errD_real + aux_errD_real
            D_x = dis_out.data.mean()

            # compute the current classification accuracy
            accuracy = compute_acc(aux_out, aux_label)
        if(curr_iter) % DISP_PER_TIMES == 0:            
            print('Val_Loss_D: %.4f, D(x): %.4f D(x_val): %.4f,  Val_Acc: %.4f (%.4f)'
                  % (errD.data, avg_loss_D, errG.data, avg_loss_G, D_x, D_G_z1, D_G_z2, accuracy, avg_loss_A))

    def train(self):

        real_label = 1
        fake_label = 0

        avg_loss_D = 0.0
        avg_loss_G = 0.0
        avg_loss_A = 0.0

        one = torch.tensor(1, dtype=torch.float)
        mone = one * -1
        if self.cuda:
            one = one.cuda(self.cuda_index)
            mone = mone.cuda(self.cuda_index)
            
        for epoch in range(self.epochs):
            self.D.train()
            self.G.train()
            for i, batch in enumerate(train_loader):
                for p in self.D.parameters():
                    p.requires_grad = True
                #Update Discriminator
                images, labels = batch

                if len(images) != self.batch_size:
                    continue

                # Update tensors and variables
                images, aux_label = self.get_torch_variable(images), self.get_torch_variable(labels)
                dis_label = self.get_torch_variable(torch.zeros((self.batch_size,)).fill_(1))

                # Zero networks gradients
                self.D.zero_grad()
                self.G.zero_grad()
                
                # Enter real images
                dis_out, aux_out = self.D(images)
                dis_errD_real = self.dis_criterion(dis_out.squeeze(1), dis_label)
                aux_errD_real = self.aux_criterion(aux_out.squeeze(1), aux_label)
                errD_real = dis_errD_real + aux_errD_real
                #errD_real = errD_real.mean()
                errD_real.backward()
                D_x = dis_out.data.mean()

                # compute the current classification accuracy
                accuracy = compute_acc(aux_out, aux_label)

                # Prepare input for fake images
                z = torch.randn(self.batch_size, self.z_dim, device=self.device)
                z = self.get_torch_variable(z).view(self.batch_size, self.z_dim, 1, 1)
                aux_label = torch.randint(0, n_classes, (batch_size,))
                aux_label = self.get_torch_variable(aux_label).long()

                # Input fake data
                fake_imgs = self.G(z, aux_label)
                dis_label = dis_label.data.fill_(0)
                dis_out, aux_out = self.D(fake_imgs.detach())
                dis_errD_fake = self.dis_criterion(dis_out.squeeze(1), dis_label)
                aux_errD_fake = self.aux_criterion(aux_out.squeeze(1), aux_label)
                errD_fake = dis_errD_fake #+ aux_errD_fake
                #errD_fake = errD_fake.mean()
                errD_fake.backward()
                D_G_z1 = dis_out.data.mean()
                errD = errD_real + errD_fake
                self.d_optimizer.step()

                # Train with gradient penalty
                gradient_penalty = self.calculate_gradient_penalty(images.data, fake_imgs.data)
                self.D.zero_grad()
                self.G.zero_grad()
                gradient_penalty.backward()
                self.d_optimizer.step()
                                                
                for p in self.D.parameters():
                    p.requires_grad = False
                # Update Generator
                self.G.zero_grad()
                self.D.zero_grad()
                dis_label = dis_label.data.fill_(1)
                dis_out, aux_out = self.D(fake_imgs)
                dis_errG = self.dis_criterion(dis_out.squeeze(1), dis_label)
                aux_errG = self.aux_criterion(aux_out.squeeze(1), aux_label)
                errG = dis_errG + aux_errG
                errG.backward()
                D_G_z2 = dis_out.data.mean()
                self.g_optimizer.step()

                # compute the average loss
                curr_iter = epoch * len(train_loader) + i
                all_loss_G = avg_loss_G * curr_iter
                all_loss_D = avg_loss_D * curr_iter
                all_loss_A = avg_loss_A * curr_iter
                all_loss_G += errG.data
                all_loss_D += errD.data
                all_loss_A += accuracy
                avg_loss_G = all_loss_G / (curr_iter + 1)
                avg_loss_D = all_loss_D / (curr_iter + 1)
                avg_loss_A = all_loss_A / (curr_iter + 1)

                if(curr_iter) % DISP_PER_TIMES == 0:            
                    print('[%d/%d][%d/%d] Loss_D: %.4f (%.4f) Loss_G: %.4f (%.4f) D(x): %.4f D(G(z)): %.4f / %.4f Acc: %.4f (%.4f)'
                          % (epoch, self.epochs, i, len(train_loader),
                             errD.data, avg_loss_D, errG.data, avg_loss_G, D_x, D_G_z1, D_G_z2, accuracy, avg_loss_A))

                if (curr_iter) % SAVE_PER_TIMES == 0:
                    self.save_model()
    
                    if not os.path.exists('PRPD-training_result_images/'):
                        os.makedirs('PRPD-training_result_images/')
                    
                    # Denormalize images and save them in grid 8x8
                    samples = self.G(self.eval_noise, self.eval_label)
                    samples = samples.mul(0.5).add(0.5).clamp(0,1)
                    samples = samples.data.cpu()
                    grid = utils.make_grid(samples)
                    utils.save_image(grid, 'PRPD-training_result_images/img_generator_iter_{}.png'.format(str(curr_iter).zfill(3)))
            decay_gauss_std(self.D)
            self.G_error[epoch] = errG.data
            self.D_error[epoch] = errD.data
            self.D_x[epoch] = D_x
            self.D_G_z1[epoch] = D_G_z1
            self.D_G_z2[epoch] = D_G_z2
            self.accuracy[epoch] = accuracy
            self.avg_loss_D[epoch] = avg_loss_D
            self.avg_loss_G[epoch] = avg_loss_G
            self.avg_accuracy[epoch] = avg_loss_A
                    
    def save_model(self):
        torch.save(self.G.state_dict(), './PRPD_generator.pkl')
        torch.save(self.D.state_dict(), './PRPD_discriminator.pkl')
        print('Models save to ./PRPD_generator.pkl & ./PRPD_discriminator.pkl ')
    
    def calculate_gradient_penalty(self, real_images, fake_images):
        eta = torch.FloatTensor(self.batch_size,1,1,1).uniform_(0,1)
        eta = eta.expand(self.batch_size, real_images.size(1), real_images.size(2), real_images.size(3))
        if self.cuda:
            eta = eta.cuda(self.cuda_index)
        else:
            eta = eta

        interpolated = eta * real_images + ((1 - eta) * fake_images)

        if self.cuda:
            interpolated = interpolated.cuda(self.cuda_index)
        else:
            interpolated = interpolated

        # define it to calculate gradient
        interpolated = Variable(interpolated, requires_grad=True)

        # calculate probability of interpolated examples
        prob_interpolated, _ = self.D(interpolated)

        # calculate gradients of probabilities with respect to examples
        gradients = autograd.grad(outputs=prob_interpolated, inputs=interpolated,
                               grad_outputs=torch.ones(
                                   prob_interpolated.size()).cuda(self.cuda_index) if self.cuda else torch.ones(
                                   prob_interpolated.size()),
                               create_graph=True, retain_graph=True)[0]

        # flatten the gradients to it calculates norm batchwise
        gradients = gradients.view(gradients.size(0), -1)
        
        grad_penalty = ((gradients.norm(2, dim=1) - 1) ** 2).mean() * self.lambda_term
        return grad_penalty
        
    def print_network(self, model, name):
        num_params = 0
        for p in model.parameters():
            num_params += p.numel()
        print(name)
        print(model)
        print("The number of parameters: {}".format(num_params))

In [9]:
################  ONLY NECESSARY TO USE THE FIRST TIME WHEN THIS NOTEBOOK IS USED  #########################
# Dataset

trainval_folder = 'data\\PRPD_dataset\\TrainVal'
test_folder = 'data\\PRPD_dataset\\Test'

def get_dataset(path, split=''):
    images_path = []
    for dirpath, dirnames, files in os.walk(path):
        for file in files:
            path = os.path.join(dirpath, file)
            motor = os.path.basename(dirpath)
            defect = os.path.basename(os.path.dirname(dirpath))
            images_path.append((path, motor, defect, split))
    return images_path

trainval_path = os.path.join(os.path.dirname(os.getcwd()), trainval_folder)
test_path = os.path.join(os.path.dirname(os.getcwd()), test_folder)

classes = os.listdir(trainval_path)
print('Defects: {}'.format(classes))

trainval_ds = get_dataset(trainval_path, split='Train-val')
test_df = get_dataset(test_path, split='Test')

# Create pandas dataframe
df = pd.DataFrame(trainval_ds+test_df, columns=['Filename', 'Motor', 'Defect', 'Split'])
df['Class'], unique_labels = pd.factorize(df['Defect'])

Defects: ['corona', 'internal', 'surface']


In [10]:
class ImageDataset(Dataset):
    def __init__(self, df: pd.DataFrame, transform=None):
        self.df = df
        self.filenames = self.df['Filename'].values
        self.labels = self.df['Class'].values
        self.images = np.zeros((len(self.labels), 128, 165, 3), dtype= np.uint8)
        self.transform = transform
        self.getImages()

    def getImages(self):
        for idx, filename in enumerate(self.filenames):
            img = cv2.imread(filename)
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            img = cv2.resize(img, (165, 128), interpolation = cv2.INTER_AREA)
            self.images[idx] = img

    def toTensor(self, data):
        image = data.transpose((2,0,1))
        return torch.from_numpy(image)

    def normalize(self, image_arr):
        return (image_arr - image_arr.min()) / (image_arr.max() - image_arr.min())
    
    def __getitem__(self, idx):
        x = self.images[idx]
        x = self.toTensor(x)
        y = self.labels[idx]
        if self.transform:
            x = self.transform(x)
        return x,y

    def __len__(self):
        return len(self.filenames)

In [11]:
transform = transforms.Compose([transforms.ToDtype(torch.float32, scale=True),
                                    transforms.RandomRotation(degrees=(0,25)),
                                    transforms.RandomHorizontalFlip(p=0.5),
                                    transforms.RandomVerticalFlip(p=0.5),
                                    transforms.GaussianBlur(kernel_size=(3,3)),
                                    transforms.RandomAdjustSharpness(sharpness_factor=1.33),
                                    transforms.Resize((128,165))])
dataset = ImageDataset(df, transform=transform)
batch_size = 32
train_loader = DataLoader(dataset=dataset, batch_size=batch_size, shuffle=True)

In [12]:
for images, labels in train_loader:
    print(images.shape, labels.shape)
    break

torch.Size([32, 3, 128, 165]) torch.Size([32])


In [13]:
if torch.cuda.is_available():
    print("CUDA is available! You can use GPU.")
    device = torch.device("cuda")
    print(f"Device: {torch.cuda.get_device_name(device)}")
    cuda = True
else:
    print("CUDA is not available. Using CPU.")
    device = torch.device("cpu")
    cuda = False

CUDA is available! You can use GPU.
Device: NVIDIA GeForce RTX 2060 with Max-Q Design


In [14]:
#n_channels, z_dim, n_classes, n_embedding, epochs, batch_size
n_channels=3
z_dim = 128
n_classes = 3
n_embedding = 64
epochs = 600
d_iter = 4
std = 0.1
decay = 0.995

SAVE_PER_TIMES = 100
DISP_PER_TIMES = 50

model = ACGAN(n_channels, z_dim, n_classes, n_embedding, epochs, batch_size, device, cuda, std, decay)
model.train()

ACGAN Model Initialization
Generator
Generator(
  (embedding): Sequential(
    (0): Embedding(3, 128)
    (1): GaussianNoise()
  )
  (fc_latent): Sequential(
    (0): Linear(in_features=128, out_features=400, bias=False)
  )
  (block_c1): Sequential(
    (0): ConvTranspose2d(400, 256, kernel_size=(4, 4), stride=(4, 4), bias=False)
    (1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
  )
  (block_c2): Sequential(
    (0): ConvTranspose2d(256, 128, kernel_size=(8, 8), stride=(4, 4), padding=(2, 2), bias=False)
    (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
  )
  (block_c3): Sequential(
    (0): ConvTranspose2d(128, 64, kernel_size=(8, 8), stride=(4, 4), padding=(2, 2), bias=False)
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
  )
  (block_c4): Sequential(
    (0): ConvTranspose2d(64, 

C:\Users\jazg2\miniconda3\envs\RL-env\Lib\site-packages\torch\autograd\graph.py:744: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at C:\cb\pytorch_1000000000000\work\aten\src\ATen\native\cudnn\Conv_v8.cpp:919.)
  return Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass


[0/600][0/30] Loss_D: 1.0724 (1.0724) Loss_G: 0.3688 (0.3688) D(x): 0.5032 D(G(z)): 0.5128 / 0.4950 Acc: 34.3750 (34.3750)
Models save to ./PRPD_generator.pkl & ./PRPD_discriminator.pkl 
[1/600][20/30] Loss_D: 1.5821 (1.2724) Loss_G: 0.1417 (0.2550) D(x): 0.3983 D(G(z)): 0.6267 / 0.6195 Acc: 31.2500 (35.8878)
[3/600][10/30] Loss_D: 1.3686 (1.3578) Loss_G: 0.2153 (0.2210) D(x): 0.4670 D(G(z)): 0.6043 / 0.5943 Acc: 28.1250 (34.9756)
Models save to ./PRPD_generator.pkl & ./PRPD_discriminator.pkl 
[5/600][0/30] Loss_D: 1.3257 (1.3264) Loss_G: 0.0009 (0.2111) D(x): 0.5391 D(G(z)): 0.6299 / 0.7088 Acc: 34.3750 (35.4527)
[6/600][20/30] Loss_D: 1.4093 (1.3205) Loss_G: 0.2609 (0.2020) D(x): 0.5495 D(G(z)): 0.6836 / 0.5810 Acc: 43.7500 (36.8885)
Models save to ./PRPD_generator.pkl & ./PRPD_discriminator.pkl 
[8/600][10/30] Loss_D: 1.3683 (1.3124) Loss_G: 0.2302 (0.2006) D(x): 0.6036 D(G(z)): 0.7068 / 0.5664 Acc: 56.2500 (37.5047)
[10/600][0/30] Loss_D: 1.3437 (1.3091) Loss_G: 0.0923 (0.1977) D(x

In [15]:
### Evaluate after training
eval_noise = torch.randn((batch_size, z_dim, 1, 1))#np.random.normal(0, 1, (self.batch_size, self.z_dim))#torch.rand((self.batch_size, 128, 1, 1))
eval_noise = model.get_torch_variable(eval_noise)

eval_label = torch.randint(0, n_classes, (batch_size,))
eval_label = model.get_torch_variable(eval_label)

model.D.eval()
model.G.eval()

 # Denormalize images and save them in grid 8x8
samples = model.G(eval_noise, eval_label)
samples = samples.mul(0.5).add(0.5).clamp(0,1)
samples = samples.data.cpu()
grid = utils.make_grid(samples)
utils.save_image(grid, 'output_example.png')

In [16]:
# Input fake data
dis_label = model.get_torch_variable(torch.zeros((batch_size,)))
samples = model.G(eval_noise, eval_label)
dis_out, aux_out = model.D(samples.detach())
dis_errD_fake = model.dis_criterion(dis_out.squeeze(1), dis_label)
aux_errD_fake = model.aux_criterion(aux_out.squeeze(1), eval_label)
errD_fake = dis_errD_fake + aux_errD_fake

print('D(G(z)): {:.3f}'.format(dis_out.mean()))
print('Loss D: {:.3f}'.format(errD_fake))

D(G(z)): 0.503
Loss D: -0.268
